In [1]:
%cd "OneDrive - National University of Singapore"

C:\Users\wongp\OneDrive - National University of Singapore


In [2]:
%cd CS4248/Project/db/

C:\Users\wongp\OneDrive - National University of Singapore\CS4248\Project\db


In [3]:
import os
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from collections import Counter
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import nltk
from nltk.stem import WordNetLemmatizer

Convert training data into csv file

In [ ]:
import os
import pandas as pd

pathNeg = "train/neg"
files = os.listdir(pathNeg)
data = []
for file in files:
  if os.path.isfile(os.path.join(pathNeg, file)):
    id = int(file.split("_")[0])
    with open(os.path.join(pathNeg, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 0])

sizeOfNeg = 12500

pathPos = "train/pos"
files = os.listdir(pathPos)
for file in files:
  if os.path.isfile(os.path.join(pathPos, file)):
    id = int(file.split("_")[0]) + sizeOfNeg
    with open(os.path.join(pathPos, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 1])


df = pd.DataFrame(data, columns = ['id', 'text', 'score']).set_index('id').sort_index()
df.to_csv('train.csv')

Convert testing data into csv file

In [ ]:
pathNeg = "test/neg"
files = os.listdir(pathNeg)
data = []
for file in files:
  if os.path.isfile(os.path.join(pathNeg, file)):
    id = int(file.split("_")[0])
    with open(os.path.join(pathNeg, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 0])

sizeOfNeg = 12500

pathPos = "test/pos"
files = os.listdir(pathPos)
for file in files:
  if os.path.isfile(os.path.join(pathPos, file)):
    id = int(file.split("_")[0]) + sizeOfNeg
    with open(os.path.join(pathPos, file), encoding='utf8') as f:
      lines = f.read()
    data.append([id, lines, 1])


df = pd.DataFrame(data, columns = ['id', 'text', 'score']).set_index('id').sort_index()
df.to_csv('test.csv')

convert vocab into csv file

In [ ]:
with open("imdb.vocab", encoding='utf8') as f:
  vocab = []
  for line in f:
    vocab.append(line.rstrip())

vocab_df = pd.DataFrame(vocab, columns=['vocab'])
print(vocab_df)
vocab_df.to_csv('vocab_given.csv', index=False)

                   vocab
0                    the
1                    and
2                      a
3                     of
4                     to
...                  ...
89522          copywrite
89523             artbox
89524          kinky-sex
89525           urrrghhh
89526  investigator-like

[89527 rows x 1 columns]


In [4]:
from nltk.tokenize import word_tokenize

def tokenizeReview(review):
  review = review.replace('.', ' . ')
  tokens = word_tokenize(review.lower())
  return tokens

Get vocab from training set

In [ ]:
from collections import Counter
reviews = train_df['text'].tolist()
reviews = list(map(tokenizeReview, reviews))
reviews = [item for review in reviews for item in review]
vdict = Counter(reviews).most_common()

In [ ]:
vocabs = []
for pair in vdict:
  vocabs.append(pair[0])

vocab_df = pd.DataFrame(vocabs, columns=['vocab'])
vocab_df.to_csv('vocab.csv', index=False)
print(vocab_df)

              vocab
0               the
1                 .
2                 ,
3               and
4                 a
...             ...
99203    people-and
99204         padre
99205   unmatchable
99206  scenery-wise
99207  non-teenager

[99208 rows x 1 columns]


Read training data and testing data from csv file (START HERE after initialization)

In [5]:
train_df = pd.read_csv('train/train.csv')
test_df = pd.read_csv('test/test.csv')
vocab_df = pd.read_csv('vocab.csv')
print(vocab_df)

              vocab
0               the
1                 .
2                 ,
3               and
4                 a
...             ...
99203    people-and
99204         padre
99205   unmatchable
99206  scenery-wise
99207  non-teenager

[99208 rows x 1 columns]


In [ ]:
vocab_dict = pd.Series(vocab_df.index + 1, index=vocab_df.vocab).to_dict()
print(vocab_dict)

In [7]:
def convertToIndex(review):
  vec = []
  for word in review:
    if word in vocab_dict:
      vec.append(int(vocab_dict[word]))
    else:
      vec.append(0)
  return vec

In [8]:
train_df['tokens'] = train_df['text'].apply(tokenizeReview)

In [9]:
train_df['vec'] = train_df['tokens'].apply(convertToIndex)

In [11]:
train_df

,id,text,score,tokens,vec
0,0,Story of a man who has unnatural feelings for ...,0,"[story, of, a, man, who, has, unnatural, feeli...","[76, 6, 5, 142, 46, 56, 7518, 1387, 21, 5, 451..."
1,1,Robert DeNiro plays the most unbelievably inte...,0,"[robert, deniro, plays, the, most, unbelievabl...","[649, 5006, 307, 1, 106, 3769, 1076, 9203, 6, ..."
2,2,"I saw the capsule comment said ""great acting.""...",0,"[i, saw, the, capsule, comment, said, ``, grea...","[15, 226, 1, 9586, 906, 311, 28, 102, 129, 2, ..."
3,3,If I had not read Pat Barker's 'Union Street' ...,0,"[if, i, had, not, read, pat, barker, 's, 'unio...","[57, 15, 78, 32, 338, 3189, 7720, 18, 34296, 8..."
4,4,This fanciful horror flick has Vincent Price p...,0,"[this, fanciful, horror, flick, has, vincent, ...","[16, 16016, 213, 512, 56, 3154, 1789, 397, 5, ..."
...,...,...,...,...,...
24995,24995,What's Good About It: Some inventive and genui...,1,"[what, 's, good, about, it, :, some, inventive...","[60, 18, 62, 55, 13, 87, 61, 4388, 4, 2028, 93..."
24996,24996,For years we've been watching every horror fil...,1,"[for, years, we, 've, been, watching, every, h...","[21, 168, 82, 155, 93, 163, 188, 213, 25, 17, ..."
24997,24997,If you haven't already seen this movie of Mary...,1,"[if, you, have, n't, already, seen, this, movi...","[57, 29, 37, 31, 466, 126, 16, 23, 6, 13586, 4..."
24998,24998,this movie is the best movie ever it has a lot...,1,"[this, movie, is, the, best, movie, ever, it, ...","[16, 23, 8, 1, 133, 23, 138, 13, 56, 5, 186, 6..."


In [ ]:
print('Maximum review length: {}'.format(
len(max((train_df['tokens']), key=len))))
print('Minimum review length: {}'.format(
len(min((train_df['tokens']), key=len))))

Maximum review length: 2820
Minimum review length: 11


Lemmatizer

In [12]:
wordnet_lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('omw-1.4')

def lemmatizer(reviewTokens):
  punctuations="?:!.,;"
  for word in reviewTokens:
    if word in punctuations:
      reviewTokens.remove(word)
  reviewLemm = []
  for word in reviewTokens:
    newWord = wordnet_lemmatizer.lemmatize(word, pos="v")
    reviewLemm.append(newWord)

  return reviewLemm


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wongp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\wongp\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [13]:
train_df['lemmTokens'] = train_df['tokens'].apply(lemmatizer)

In [14]:
train_df

,id,text,score,tokens,vec,lemmTokens
0,0,Story of a man who has unnatural feelings for ...,0,"[story, of, a, man, who, has, unnatural, feeli...","[76, 6, 5, 142, 46, 56, 7518, 1387, 21, 5, 451...","[story, of, a, man, who, have, unnatural, feel..."
1,1,Robert DeNiro plays the most unbelievably inte...,0,"[robert, deniro, plays, the, most, unbelievabl...","[649, 5006, 307, 1, 106, 3769, 1076, 9203, 6, ...","[robert, deniro, play, the, most, unbelievably..."
2,2,"I saw the capsule comment said ""great acting.""...",0,"[i, saw, the, capsule, comment, said, ``, grea...","[15, 226, 1, 9586, 906, 311, 28, 102, 129, 2, ...","[i, saw, the, capsule, comment, say, ``, great..."
3,3,If I had not read Pat Barker's 'Union Street' ...,0,"[if, i, had, not, read, pat, barker, 's, 'unio...","[57, 15, 78, 32, 338, 3189, 7720, 18, 34296, 8...","[if, i, have, not, read, pat, barker, 's, 'uni..."
4,4,This fanciful horror flick has Vincent Price p...,0,"[this, fanciful, horror, flick, has, vincent, ...","[16, 16016, 213, 512, 56, 3154, 1789, 397, 5, ...","[this, fanciful, horror, flick, have, vincent,..."
...,...,...,...,...,...,...
24995,24995,What's Good About It: Some inventive and genui...,1,"[what, 's, good, about, it, some, inventive, a...","[60, 18, 62, 55, 13, 87, 61, 4388, 4, 2028, 93...","[what, 's, good, about, it, some, inventive, a..."
24996,24996,For years we've been watching every horror fil...,1,"[for, years, we, 've, been, watching, every, h...","[21, 168, 82, 155, 93, 163, 188, 213, 25, 17, ...","[for, years, we, 've, be, watch, every, horror..."
24997,24997,If you haven't already seen this movie of Mary...,1,"[if, you, have, n't, already, seen, this, movi...","[57, 29, 37, 31, 466, 126, 16, 23, 6, 13586, 4...","[if, you, have, n't, already, see, this, movie..."
24998,24998,this movie is the best movie ever it has a lot...,1,"[this, movie, is, the, best, movie, ever, it, ...","[16, 23, 8, 1, 133, 23, 138, 13, 56, 5, 186, 6...","[this, movie, be, the, best, movie, ever, it, ..."


In [ ]:
from collections import Counter
reviewsLem = train_df['lemmTokens'].tolist()
# reviewsLem = list(map(tokenizeReview, reviewsLem))
reviewsLem = [item for review in reviewsLem for item in review]
vdictLem = Counter(reviewsLem).most_common()
#print(vdictLem)

vdictLemToIdx = {}
idx = 1
for pair in vdictLem:
  vdictLemToIdx[pair[0]] = idx
  idx = idx + 1
print(vdictLemToIdx)

In [16]:
def convertToIndex(review):
  vec = []
  for word in review:
    if word in vdictLemToIdx:
      vec.append(int(vdictLemToIdx[word]))
    else:
      vec.append(0)
  return vec

In [17]:
train_df['vecLem'] = train_df['lemmTokens'].apply(convertToIndex)

In [18]:
train_df

,id,text,score,tokens,vec,lemmTokens,vecLem
0,0,Story of a man who has unnatural feelings for ...,0,"[story, of, a, man, who, has, unnatural, feeli...","[76, 6, 5, 142, 46, 56, 7518, 1387, 21, 5, 451...","[story, of, a, man, who, have, unnatural, feel...","[76, 5, 4, 135, 43, 17, 6484, 145, 20, 4, 2990..."
1,1,Robert DeNiro plays the most unbelievably inte...,0,"[robert, deniro, plays, the, most, unbelievabl...","[649, 5006, 307, 1, 106, 3769, 1076, 9203, 6, ...","[robert, deniro, play, the, most, unbelievably...","[654, 4427, 107, 1, 104, 3383, 1052, 7869, 5, ..."
2,2,"I saw the capsule comment said ""great acting.""...",0,"[i, saw, the, capsule, comment, said, ``, grea...","[15, 226, 1, 9586, 906, 311, 28, 102, 129, 2, ...","[i, saw, the, capsule, comment, say, ``, great...","[13, 228, 1, 8022, 417, 88, 27, 99, 106, 27, 1..."
3,3,If I had not read Pat Barker's 'Union Street' ...,0,"[if, i, had, not, read, pat, barker, 's, 'unio...","[57, 15, 78, 32, 338, 3189, 7720, 18, 34296, 8...","[if, i, have, not, read, pat, barker, 's, 'uni...","[53, 13, 17, 31, 266, 2761, 6646, 16, 28419, 8..."
4,4,This fanciful horror flick has Vincent Price p...,0,"[this, fanciful, horror, flick, has, vincent, ...","[16, 16016, 213, 512, 56, 3154, 1789, 397, 5, ...","[this, fanciful, horror, flick, have, vincent,...","[14, 13459, 215, 420, 17, 2872, 1554, 107, 4, ..."
...,...,...,...,...,...,...,...
24995,24995,What's Good About It: Some inventive and genui...,1,"[what, 's, good, about, it, some, inventive, a...","[60, 18, 62, 55, 13, 87, 61, 4388, 4, 2028, 93...","[what, 's, good, about, it, some, inventive, a...","[56, 16, 59, 52, 11, 57, 3881, 3, 1876, 925, 1..."
24996,24996,For years we've been watching every horror fil...,1,"[for, years, we, 've, been, watching, every, h...","[21, 168, 82, 155, 93, 163, 188, 213, 25, 17, ...","[for, years, we, 've, be, watch, every, horror...","[20, 164, 79, 149, 2, 65, 186, 215, 18, 15, 10..."
24997,24997,If you haven't already seen this movie of Mary...,1,"[if, you, have, n't, already, seen, this, movi...","[57, 29, 37, 31, 466, 126, 16, 23, 6, 13586, 4...","[if, you, have, n't, already, see, this, movie...","[53, 28, 17, 30, 480, 44, 14, 22, 5, 11464, 3,..."
24998,24998,this movie is the best movie ever it has a lot...,1,"[this, movie, is, the, best, movie, ever, it, ...","[16, 23, 8, 1, 133, 23, 138, 13, 56, 5, 186, 6...","[this, movie, be, the, best, movie, ever, it, ...","[14, 22, 2, 1, 129, 22, 132, 11, 17, 4, 156, 5..."


In [19]:
max_words = 500
dataset = sequence.pad_sequences(train_df['vecLem'], maxlen=max_words)
y = train_df['score']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(dataset, y, test_size=0.3, stratify=y, random_state = 42)


In [20]:
from gensim.models import Word2Vec
import gensim.downloader as api

v2w_model = api.load('glove-wiki-gigaword-50')
print(v2w_model['computer'])

[ 0.079084 -0.81504   1.7901    0.91653   0.10797  -0.55628  -0.84427
 -1.4951    0.13418   0.63627   0.35146   0.25813  -0.55029   0.51056
  0.37409   0.12092  -1.6166    0.83653   0.14202  -0.52348   0.73453
  0.12207  -0.49079   0.32533   0.45306  -1.585    -0.63848  -1.0053
  0.10454  -0.42984   3.181    -0.62187   0.16819  -1.0139    0.064058
  0.57844  -0.4556    0.73783   0.37203  -0.57722   0.66441   0.055129
  0.037891  1.3275    0.30991   0.50697   1.2357    0.1274   -0.11434
  0.20709 ]


In [21]:
embedding_dim = 100
vocab_len = len(vdictLemToIdx) + 1
embed_vector_len = v2w_model['computer'].shape[0]
emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, i in vdictLemToIdx.items():
  if word in v2w_model:
    embedding_vector = v2w_model.get_vector(word)
    emb_matrix[i] = embedding_vector


In [22]:
emb_matrix[1]

array([ 4.18000013e-01,  2.49679998e-01, -4.12420005e-01,  1.21699996e-01,
        3.45270008e-01, -4.44569997e-02, -4.96879995e-01, -1.78619996e-01,
       -6.60229998e-04, -6.56599998e-01,  2.78430015e-01, -1.47670001e-01,
       -5.56770027e-01,  1.46579996e-01, -9.50950012e-03,  1.16579998e-02,
        1.02040000e-01, -1.27920002e-01, -8.44299972e-01, -1.21809997e-01,
       -1.68009996e-02, -3.32789987e-01, -1.55200005e-01, -2.31309995e-01,
       -1.91809997e-01, -1.88230002e+00, -7.67459989e-01,  9.90509987e-02,
       -4.21249986e-01, -1.95260003e-01,  4.00710011e+00, -1.85939997e-01,
       -5.22870004e-01, -3.16810012e-01,  5.92130003e-04,  7.44489999e-03,
        1.77780002e-01, -1.58969998e-01,  1.20409997e-02, -5.42230010e-02,
       -2.98709989e-01, -1.57490000e-01, -3.47579986e-01, -4.56370004e-02,
       -4.42510009e-01,  1.87849998e-01,  2.78489990e-03, -1.84110001e-01,
       -1.15139998e-01, -7.85809994e-01])

In [ ]:
print(vocab_len)

88182


In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=max_words, weights=[emb_matrix], trainable=False)

embedding_size=32
model=Sequential()
model.add(embedding_layer)
model.add(GRU(units=50, return_sequences = True, input_shape=(X_train.shape[1],1), activation='tanh'))
#model.add(LSTM(100))
model.add(Dropout(0.6))
model.add(GRU(units=50, input_shape=(X_train.shape[1],1), activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 50)           4409100   
                                                                 
 gru (GRU)                   (None, 500, 50)           15300     
                                                                 
 dropout (Dropout)           (None, 500, 50)           0         
                                                                 
 gru_1 (GRU)                 (None, 50)                15300     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 4,439,751
Trainable params: 30,651
Non-trainable params: 4,409,100
_________________________________________________________________
None


In [ ]:
batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

Epoch 1/5
547/547 [==============================] - 131s 235ms/step - loss: 0.6336 - accuracy: 0.6287
Epoch 2/5
547/547 [==============================] - 128s 234ms/step - loss: 0.4394 - accuracy: 0.7997
Epoch 3/5
547/547 [==============================] - 129s 237ms/step - loss: 0.3768 - accuracy: 0.8320
Epoch 4/5
547/547 [==============================] - 125s 228ms/step - loss: 0.3498 - accuracy: 0.8476
Epoch 5/5
547/547 [==============================] - 129s 235ms/step - loss: 0.3359 - accuracy: 0.8551


In [ ]:
scores = model.evaluate(X_val, y_val, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.8587999939918518


In [ ]:
model.save('basic_model')

INFO:tensorflow:Assets written to: basic_gru_model\assets


INFO:tensorflow:Assets written to: basic_gru_model\assets


**Without Glove word embedding**

In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU
embedding_size=32
model=Sequential()
model.add(Embedding(vocab_len, embedding_size, input_length=max_words))
model.add(GRU(units=50, input_shape=(X_train.shape[1],1), activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 32)           2821824   
                                                                 
 gru_2 (GRU)                 (None, 50)                12600     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 2,834,475
Trainable params: 2,834,475
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

Epoch 1/5
547/547 [==============================] - 73s 131ms/step - loss: 0.4788 - accuracy: 0.7517
Epoch 2/5
547/547 [==============================] - 74s 135ms/step - loss: 0.2361 - accuracy: 0.9089
Epoch 3/5
547/547 [==============================] - 73s 134ms/step - loss: 0.1233 - accuracy: 0.9581
Epoch 4/5
547/547 [==============================] - 73s 133ms/step - loss: 0.0666 - accuracy: 0.9804
Epoch 5/5
547/547 [==============================] - 73s 133ms/step - loss: 0.0372 - accuracy: 0.9881


In [ ]:
scores = model.evaluate(X_val, y_val, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.8755999803543091


In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU
embedding_size=32
model=Sequential()
model.add(Embedding(vocab_len, embedding_size, input_length=max_words))
model.add(GRU(units=50, return_sequences = True, input_shape=(X_train.shape[1],1), activation='tanh'))
#model.add(LSTM(100))
model.add(Dropout(0.6))
model.add(GRU(units=50, input_shape=(X_train.shape[1],1), activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

scores = model.evaluate(X_val, y_val, verbose=0)
print('Test accuracy:', scores[1])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 32)           2821824   
                                                                 
 gru_3 (GRU)                 (None, 500, 50)           12600     
                                                                 
 dropout_1 (Dropout)         (None, 500, 50)           0         
                                                                 
 gru_4 (GRU)                 (None, 50)                15300     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 2,849,775
Trainable params: 2,849,775
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
547/547 [====================

In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=max_words, weights=[emb_matrix], trainable=True)

embedding_size=32
model=Sequential()
model.add(embedding_layer)
model.add(GRU(units=50, return_sequences = True, input_shape=(X_train.shape[1],1), activation='tanh'))
#model.add(LSTM(100))
model.add(Dropout(0.6))
model.add(GRU(units=50, input_shape=(X_train.shape[1],1), activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

scores = model.evaluate(X_val, y_val, verbose=0)
print('Test accuracy:', scores[1])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 50)           4409100   
                                                                 
 gru_5 (GRU)                 (None, 500, 50)           15300     
                                                                 
 dropout_2 (Dropout)         (None, 500, 50)           0         
                                                                 
 gru_6 (GRU)                 (None, 50)                15300     
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 4,439,751
Trainable params: 4,439,751
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
547/547 [====================

In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=max_words, weights=[emb_matrix], trainable=True)

embedding_size=32
model=Sequential()
model.add(embedding_layer)
model.add(GRU(units=50, input_shape=(X_train.shape[1],1), activation='tanh'))
#model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

scores = model.evaluate(X_val, y_val, verbose=0)
print('Test accuracy:', scores[1])

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 500, 50)           4409100   
                                                                 
 gru_7 (GRU)                 (None, 50)                15300     
                                                                 
 dense_4 (Dense)             (None, 1)                 51        
                                                                 
Total params: 4,424,451
Trainable params: 4,424,451
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
547/547 [==============================] - 78s 140ms/step - loss: 0.5365 - accuracy: 0.7171
Epoch 2/5
547/547 [==============================] - 77s 140ms/step - loss: 0.2720 - accuracy: 0.8893
Epoch 3/5
547/547 [==============================] - 77s 140ms/step - loss: 0.1542 - accuracy: 0.94

In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=max_words, weights=[emb_matrix], trainable=False)

embedding_size=32
model=Sequential()
model.add(embedding_layer)
model.add(GRU(units=50, input_shape=(X_train.shape[1],1), activation='tanh'))
#model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

scores = model.evaluate(X_val, y_val, verbose=0)
print('Test accuracy:', scores[1])

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 500, 50)           4409100   
                                                                 
 gru_8 (GRU)                 (None, 50)                15300     
                                                                 
 dense_5 (Dense)             (None, 1)                 51        
                                                                 
Total params: 4,424,451
Trainable params: 15,351
Non-trainable params: 4,409,100
_________________________________________________________________
None
Epoch 1/5
547/547 [==============================] - 68s 118ms/step - loss: 0.5859 - accuracy: 0.6735
Epoch 2/5
547/547 [==============================] - 64s 117ms/step - loss: 0.4146 - accuracy: 0.8137
Epoch 3/5
547/547 [==============================] - 65s 119ms/step - loss: 0.3698 - accuracy:

# Testing Data

In [25]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=max_words, weights=[emb_matrix], trainable=True)

embedding_size=32
model=Sequential()
model.add(embedding_layer)
model.add(GRU(units=50, return_sequences = True, input_shape=(dataset.shape[1],1), activation='tanh'))
#model.add(LSTM(100))
model.add(Dropout(0.6))
model.add(GRU(units=50, input_shape=(dataset.shape[1],1), activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 50)           4409100   
                                                                 
 gru_2 (GRU)                 (None, 500, 50)           15300     
                                                                 
 dropout_1 (Dropout)         (None, 500, 50)           0         
                                                                 
 gru_3 (GRU)                 (None, 50)                15300     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 4,439,751
Trainable params: 4,439,751
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
batch_size=32
model.fit(dataset, y, epochs = 5, batch_size=batch_size, verbose = 'auto')

Epoch 1/5
782/782 [==============================] - 208s 263ms/step - loss: 0.4791 - accuracy: 0.7587
Epoch 2/5
782/782 [==============================] - 203s 259ms/step - loss: 0.2394 - accuracy: 0.9064
Epoch 3/5
782/782 [==============================] - 205s 262ms/step - loss: 0.1367 - accuracy: 0.9517
Epoch 4/5
782/782 [==============================] - 203s 260ms/step - loss: 0.0673 - accuracy: 0.9792
Epoch 5/5
782/782 [==============================] - 204s 261ms/step - loss: 0.0322 - accuracy: 0.9908


In [27]:
test_df['tokens'] = test_df['text'].apply(tokenizeReview)
test_df['lemmTokens'] = test_df['tokens'].apply(lemmatizer)
test_df['vecLem'] = test_df['lemmTokens'].apply(convertToIndex)
test_x = sequence.pad_sequences(test_df['vecLem'], maxlen=max_words)
test_x

array([[   0,    0,    0, ...,   42,  585,   12],
       [   0,    0,    0, ...,  293,   16,  299],
       [   0,    0,    0, ...,  431,   81,   55],
       ...,
       [   0,    0,    0, ...,  698,  772, 2313],
       [   0,    0,    0, ...,  396,   37,   58],
       [   0,    0,    0, ...,  710, 2189,   55]])

In [28]:
test_y = test_df['score']
scores = model.evaluate(test_x, test_y, verbose=0)
print('Test accuracy:', scores[1])
pred = model.predict(test_x)
from sklearn.metrics import accuracy_score
pred_labels = []
for i in pred:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(test_y,pred_labels))

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(test_y,pred_labels)
print(cm)

test_df['pred'] = pred_labels
test_df
test_df.to_csv("test_df_pred_with_lemma__trainable_true_gru.csv")

Test accuracy: 0.8653600215911865
Accuracy of prediction on test set :  0.86536
[[10983  1517]
 [ 1849 10651]]


**trainable false**

In [29]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=max_words, weights=[emb_matrix], trainable=False)

embedding_size=32
model=Sequential()
model.add(embedding_layer)
model.add(GRU(units=50, return_sequences = True, input_shape=(dataset.shape[1],1), activation='tanh'))
#model.add(LSTM(100))
model.add(Dropout(0.6))
model.add(GRU(units=50, input_shape=(dataset.shape[1],1), activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 50)           4409100   
                                                                 
 gru_4 (GRU)                 (None, 500, 50)           15300     
                                                                 
 dropout_2 (Dropout)         (None, 500, 50)           0         
                                                                 
 gru_5 (GRU)                 (None, 50)                15300     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 4,439,751
Trainable params: 30,651
Non-trainable params: 4,409,100
_________________________________________________________________
None


In [30]:
batch_size=32
model.fit(dataset, y, epochs = 5, batch_size=batch_size, verbose = 'auto')

Epoch 1/5
782/782 [==============================] - 183s 232ms/step - loss: 0.5486 - accuracy: 0.7076
Epoch 2/5
782/782 [==============================] - 182s 233ms/step - loss: 0.3886 - accuracy: 0.8294
Epoch 3/5
782/782 [==============================] - 187s 239ms/step - loss: 0.3515 - accuracy: 0.8503
Epoch 4/5
782/782 [==============================] - 183s 234ms/step - loss: 0.3281 - accuracy: 0.8588
Epoch 5/5
782/782 [==============================] - 182s 233ms/step - loss: 0.3115 - accuracy: 0.8695


In [31]:
test_y = test_df['score']
scores = model.evaluate(test_x, test_y, verbose=0)
print('Test accuracy:', scores[1])
pred = model.predict(test_x)
from sklearn.metrics import accuracy_score
pred_labels = []
for i in pred:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(test_y,pred_labels))

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(test_y,pred_labels)
print(cm)

test_df['pred'] = pred_labels
test_df
test_df.to_csv("test_df_pred_with_lemma__trainable_false_gru.csv")

Test accuracy: 0.8665199875831604
Accuracy of prediction on test set :  0.86652
[[10948  1552]
 [ 1785 10715]]
